<a href="https://colab.research.google.com/github/pretty-neat-org/siap/blob/main/SIAP_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/SIAP

Mounted at /content/gdrive
config.json  data_mini_10k.csv	file.csv	   in_domain_train.tsv
data.csv     file5m.zip		glove.6B.300d.txt  pytorch_model.bin


In [19]:
import pandas as pd

import time
import pickle
import os

import nltk

# download just the first time
nltk.download('all')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

Functions for preparing data

In [20]:
'''Function for converting string to wordnet tag.'''

def convert_to_wordnet_tag(string_tag):
    if string_tag.startswith('J'):
        return wordnet.ADJ
    elif string_tag.startswith('V'):
        return wordnet.VERB
    elif string_tag.startswith('N'):
        return wordnet.NOUN
    elif string_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

'''Function for lemmatizing one comment(sentence).'''

def lemmatize_comment(comment, lemmatizer):
    nltk_with_tags = nltk.pos_tag(nltk.word_tokenize(comment))  # word + tag (string_tag in function above)
    result_words = []
    for word, tag in map(lambda x: (x[0], convert_to_wordnet_tag(x[1])), nltk_with_tags): # mapping to right shape
        if tag is None:
            result_words.append(word)
        else:
            result_words.append(lemmatizer.lemmatize(word, tag))
    return " ".join(result_words)

'''Function for lemmatizing whole corpus(multiple sentences).'''

def lemmatize_corpus(comments, lemmatizer):
    lemmatized_comments = []
    for q in comments:
        new_q = lemmatize_comment(q, lemmatizer)
        lemmatized_comments.append(new_q)
    return lemmatized_comments

def lemmatization(comments, lemmatizer):
    if os.path.isfile('gdrive/MyDrive/SIAP/serialized/lemmatized_comments.pkl'):
        with open('gdrive/MyDrive/SIAP/serialized/lemmatized_comments.pkl', 'rb') as f:
            deserialized_lemmatized_comments = pickle.load(f)
        if deserialized_lemmatized_comments is not None:
            lemmatized_comments = deserialized_lemmatized_comments
    else:
        print("Lemmatization...")
        start_time = time.time()
        lemmatized_comments = lemmatize_corpus(comments, lemmatizer)
        print("Lemmatization time: " + str(time.time() - start_time))
        with open('gdrive/MyDrive/SIAP/serialized/lemmatized_comments.pkl', 'wb') as f:
            pickle.dump(lemmatized_comments, f)

    return lemmatized_comments

Function for TF - IDF calculations

In [21]:
def tf_idf_calculation(comments, vectorizer):
    tf_idf_vectors = vectorizer.fit_transform(comments)
    vocab = vectorizer.get_feature_names()
    return tf_idf_vectors, vocab

Balancing dataset

In [22]:
def balance_dataset(df):
    print(df.columns)
    # Jaaaako bitno! U datasetu imaju null vrste, ovim ih izbacujemo
    df = df.dropna(subset=['body'])


    controverse_comments = df[df['controversiality'] == 1]
    non_contraverse = df[:11000]

    df = controverse_comments.append(non_contraverse, ignore_index=True)
    return df

Function for testing model on new comment

In [23]:
def test_model_on_new_comment(novi_komentar, model):
    novi_komentar_lemmatized = lemmatize_comment(novi_komentar, lemmatizer)
    new_comment_vectorized_1 = vectorizer_1.transform([novi_komentar_lemmatized])


    prediction = model.predict(new_comment_vectorized_1)
    print(prediction)

MAIN - praparing data for model fitting and fitting a model

In [29]:
    df = pd.read_csv('gdrive/MyDrive/SIAP/file.csv')
    df = balance_dataset(df)

    df['body'] = df['body'].str.replace(r'http\S+', '')
    df['body'] = df['body'].str.replace(r'[^\w\s]', '')
    df['body'] = df['body'].str.replace(r"([^\x00-\x7F])+", '')
    df['body'] = df['body'].str.lower()

    # lista sa oznakom kontroverznosti komentara 0 i 1
    controversialites = df['controversiality']
    controverse_comments = df[df['controversiality'] == 1]


    # svi komentari kojima odgovaraju oznake kontroverznosti iz liste gore
    comments = df['body']
    all_comments = comments.to_numpy()

    #all_comments = all_comments[~pd.isnull(all_comments)] # izbacivanje null vrednosti, ali ne treba jer smo vec izbacili iz df

    # Definisemo lemmatizer, mozemo da probamo i neki drugi
    lemmatizer = WordNetLemmatizer()
    lemmatized_comment = lemmatization(all_comments, lemmatizer)

    # Definisemo vectorizer
    vectorizer_1 = TfidfVectorizer()
    tf_idf_vectors_1, _ = tf_idf_calculation(lemmatized_comment, vectorizer_1)

    # SVM
    # Deljenje dataseta na obucavajuci i trening
    X_train, X_test, y_train, y_test = train_test_split(tf_idf_vectors_1, controversialites, test_size=0.3,
                                                        random_state=109)  # 70% training and 30% test

    # Kreiranje svm Classifier-a
    clf = svm.SVC(kernel='linear')  # Linear Kernel

    # Treniranje modela za trening podacima
    clf.fit(X_train, y_train)

    # Predict the response for test dataset
    y_pred = clf.predict(X_test)

    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # Model Precision: what percentage of positive tuples are labeled as such?
    print("Precision:", metrics.precision_score(y_test, y_pred, zero_division = True))

    # Model Recall: what percentage of positive tuples are labelled as such?
    print("Recall:", metrics.recall_score(y_test, y_pred, zero_division = True))

Index(['created_utc', 'ups', 'subreddit_id', 'link_id', 'name', 'score_hidden',
       'author_flair_css_class', 'author_flair_text', 'subreddit', 'id',
       'removal_reason', 'gilded', 'downs', 'archived', 'author', 'score',
       'retrieved_on', 'body', 'distinguished', 'edited', 'controversiality',
       'parent_id'],
      dtype='object')
Accuracy: 0.6601257090295876
Precision: 0.6656617425384384
Recall: 0.6658624849215923


Testing on new question


In [26]:
  novi_komentar = all_comments[1]
  print(novi_komentar)
  test_model_on_new_comment(novi_komentar, clf)

  novi_komentar = all_comments[15000]
  print(novi_komentar)
  test_model_on_new_comment(novi_komentar, clf)

I honestly wouldn't have believed it if I didn't live it. She made his life hell and I had a front row seat. I'm just glad I had a front row seat to her confession and firing. :)
[1]
There are many who probably do not. And nothing protects against walking STDs. So, don't be silly. Wrap your willy.
[0]
